In [1]:
import pandas as pd
import numpy as np
import json
import geopandas as gpd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
# Read 
india = json.load(open('state/india_states.geojson'))

#  Read in the district-level data
districts = gpd.read_file('district/india_district.geojson')

# # Read in the rainfall data
df = pd.read_excel('india_-_monthly_rainfall_data_-_1901_to_2002.xlsx')

In [3]:
# combine ladakh and jammu and kashmir area into one in the geojson file
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
for i in range(len(india['features'])):
    if india['features'][i]['properties']['ST_NM'] == 'Jammu & Kashmir':
        india['features'][i]['properties']['ST_NM'] = 'Jammu and Kashmir and Ladakh'
        jk = shape(india['features'][i]['geometry'])
    if india['features'][i]['properties']['ST_NM'] == 'Ladakh':
        india['features'][i]['properties']['ST_NM'] = 'Jammu and Kashmir and Ladakh'
        ld = shape(india['features'][i]['geometry'])

jkld = cascaded_union([jk, ld])
# remove ladakh and jammu and kashmir area from the geojson file
india['features'] = [i for i in india['features'] if i['properties']['ST_NM'] != 'Jammu and Kashmir and Ladakh']
# add the combined ladakh and jammu and kashmir area to the geojson file
india['features'].append({'type': 'Feature', 'properties': {'ST_NM': 'Jammu & Kashmir'}, 'geometry': mapping(jkld)})

# do same for Andhra Pradesh and Telangana
for i in range(len(india['features'])):
    if india['features'][i]['properties']['ST_NM'] == 'Andhra Pradesh':
        india['features'][i]['properties']['ST_NM'] = 'Andhra Pradesh and Telangana'
        ap = shape(india['features'][i]['geometry'])
    if india['features'][i]['properties']['ST_NM'] == 'Telangana':
        india['features'][i]['properties']['ST_NM'] = 'Andhra Pradesh and Telangana'
        ts = shape(india['features'][i]['geometry'])
    
apts = cascaded_union([ap, ts])
india['features'] = [i for i in india['features'] if i['properties']['ST_NM'] != 'Andhra Pradesh and Telangana']
india['features'].append({'type': 'Feature', 'properties': {'ST_NM': 'Andhra Pradesh'}, 'geometry': mapping(apts)})

C:\Users\mrbha\AppData\Local\Temp\ipykernel_22464\2343628423.py:12: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  jkld = cascaded_union([jk, ld])
C:\Users\mrbha\AppData\Local\Temp\ipykernel_22464\2343628423.py:27: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  apts = cascaded_union([ap, ts])


In [4]:
# create a dictionary to map the state name to a number
state_id_map = {}
for feature in india['features']:
    feature['id'] = feature['properties']['ST_NM']
    state_id_map[feature['properties']['ST_NM']] = feature['id']

In [5]:
state_id_map['Andaman & Nicobar Islands'] = state_id_map['Andaman & Nicobar']
state_id_map['Chattisgarh'] = state_id_map['Chhattisgarh']
state_id_map['Orissa'] = state_id_map['Odisha']
state_id_map['Pondicherry'] = state_id_map['Puducherry']
state_id_map['Dadra & Nagar Haveli'] = state_id_map['Dadra and Nagar Haveli and Daman and Diu']
state_id_map['Daman & Diu'] = state_id_map['Dadra and Nagar Haveli and Daman and Diu']
state_id_map['Uttaranchal'] = state_id_map['Uttarakhand']

In [6]:
df['id'] = df['State'].apply(lambda x: state_id_map[x])

In [7]:
def plot_rainfall(month, year):
    # choose data for the year 2002
    df_dynamic = df[df['Year'] == year]
    # group the data by state
    df_dynamic = df_dynamic.groupby('State')
    # calculate the mean of the rainfall for that month for each state
    df_dynamic = df_dynamic[month].mean().reset_index()
    # add the id column to the dataframe
    df_dynamic['id'] = df_dynamic['State'].apply(lambda x: state_id_map[x])
    # plot the choropleth map
    fig = px.choropleth(df_dynamic, geojson=india, locations='id', color=month, hover_name='State', hover_data=[month])
    fig.update_geos(fitbounds='locations', visible=False)
    # make the figure background transparent and text color white
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')
    return fig

In [8]:
district_id_map = {}
for i in range(len(districts)):
    district_id_map[districts['NAME_2'][i]] = districts['ID_2'][i]
    # if var2 is not None: then split var2 by , or | and add to dict
    if districts['VARNAME_2'][i] is not np.nan:
        if '|' in districts['VARNAME_2'][i]:
            for var2 in districts['VARNAME_2'][i].split('|'):
                district_id_map[var2] = districts['ID_2'][i]
        elif ',' in districts['VARNAME_2'][i]:
            for var2 in districts['VARNAME_2'][i].split(','):
                # remove spaces
                var2 = var2.strip()
                district_id_map[var2] = districts['ID_2'][i]
        else:
            district_id_map[districts['VARNAME_2'][i]] = districts['ID_2'][i]

In [9]:
district_id_map['Andaman'] = district_id_map['Andaman Islands']
district_id_map['Nicobar'] = district_id_map['Nicobar Islands']
district_id_map['Dibang Valley'] = district_id_map['Lower Dibang Valley']
district_id_map['Dhubri'] = district_id_map['Dhuburi']
district_id_map['Golapara'] = district_id_map['Goalpara']
district_id_map['Paschim Champaran'] = district_id_map['West Champaran']
district_id_map['Rajnandgaon'] = district_id_map['Raj Nandgaon']
district_id_map['Dadra & Nagar Haveli'] = district_id_map['Dadra and Nagar Haveli']
district_id_map['New Delhi'] = district_id_map['Delhi']
district_id_map['Yamunanagar'] = district_id_map['Yamuna Nagar']
district_id_map['Lahul & Spiti'] = district_id_map['Lahul and Spiti']
district_id_map['Anantanag'] = district_id_map['Anantnag (Kashmir South)']
district_id_map['Badgam'] = district_id_map['Bagdam']
district_id_map['Baramula'] = district_id_map['Baramula (Kashmir North)']
district_id_map['Kupwara'] = district_id_map['Kupwara (Muzaffarabad)']
district_id_map['Leh'] = district_id_map['Ladakh (Leh)']
district_id_map['Garwah'] = district_id_map['Garhwa']
district_id_map['Hazaribagh'] = district_id_map['Hazaribag']
district_id_map['Kodarma'] = district_id_map['Koderma']
district_id_map['Pakaur'] = district_id_map['Pakur']
district_id_map['West Singbhum'] = district_id_map['Singhbhum West']
district_id_map['East Singbhum'] = district_id_map['Singhbhum East']
district_id_map['Bangalore'] = district_id_map['Bangalore Urban']
district_id_map['Chamarajanagar'] = district_id_map['Chamrajnagar']
district_id_map['Chikmangalur'] = district_id_map['Chikmagalur']
district_id_map['Dakshina Kannada'] = district_id_map['Dakshin Kannad']
district_id_map['Uttara Kannada'] = district_id_map['Uttar Kannand']
district_id_map['Pathanamthitta'] = district_id_map['Pattanamtitta']
district_id_map['Gadchiroli'] = district_id_map['Garhchiroli']
district_id_map['Mumbai'] = district_id_map['Greater Bombay']
district_id_map['Mumbai (Suburban)'] = district_id_map['Greater Bombay']
district_id_map['Imphal East'] = district_id_map['East Imphal']
district_id_map['Imphal West'] = district_id_map['West Imphal']
district_id_map['Aizwal'] = district_id_map['Aizawl']
district_id_map['Anugul'] = district_id_map['Angul']
district_id_map['Bargarh'] = district_id_map['Baragarh']
district_id_map['Baudh'] = district_id_map['Boudh']
district_id_map['Jagatsinghapur'] = district_id_map['Jagatsinghpur']
district_id_map['Jajapur'] = district_id_map['Jajpur']
district_id_map['Nabarangapur'] = district_id_map['Nabarangpur']
district_id_map['Sonapur'] = district_id_map['Sonepur']
district_id_map['Pondicherry'] = district_id_map['Puducherry']
district_id_map['Nawanshahr'] = district_id_map['Nawan Shehar']
district_id_map['East Sikkim'] = district_id_map['East']
district_id_map['Sikkim'] = district_id_map['West Sikkim']
district_id_map['Imphal East'] = district_id_map['East Imphal']
district_id_map['Imphal'] = district_id_map['West Imphal']
district_id_map['The Nilgiris'] = district_id_map['Nilgiris']
district_id_map['Thoothukkudi'] = district_id_map['Thoothukudi']
district_id_map['Tiruchirapalli'] = district_id_map['Tiruchchirappalli']
district_id_map['Tirunelveli'] = district_id_map['Tirunelveli Kattabo']
district_id_map['Viluppuram'] = district_id_map['Villupuram']
district_id_map['Barabanki'] = district_id_map['Bara Banki']
district_id_map['Budaun'] = district_id_map['Badaun']
district_id_map['Sant Ravidas Nagar'] = district_id_map['Sant Ravi Das Nagar']
district_id_map['Shrawasti'] = district_id_map['Shravasti']
district_id_map['Dehradun'] = district_id_map['Dehra Dun']
district_id_map['Nainital'] = district_id_map['Naini Tal']
district_id_map['Rudraprayag'] = district_id_map['Rudra Prayag']
district_id_map['Burdwan'] = district_id_map['Barddhaman']
district_id_map['Cooch Behar'] = district_id_map['Kochbihar']
district_id_map['South Dinajpur'] = district_id_map['Dakshin Dinajpur']
district_id_map['Darjeeling'] = district_id_map['Darjiling']
district_id_map['Howrah'] = district_id_map['Haora']
district_id_map['Hooghly'] = district_id_map['Hugli']
district_id_map['Malda'] = district_id_map['Maldah']
district_id_map['Midnapore'] = district_id_map['West Midnapore']
district_id_map['Purulia'] = district_id_map['Puruliya']
district_id_map['Lakshadweep'] = district_id_map['Kavaratti']
district_id_map['Goa'] = district_id_map['North Goa']

In [10]:
df['ID_2'] = df['District'].apply(lambda x: district_id_map[x])
df.head()

,State,District,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,vlookup,id,ID_2
0,Andaman & Nicobar Islands,Andaman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andaman & Nicobar IslandsAndaman,Andaman & Nicobar,1
1,Andaman & Nicobar Islands,Nicobar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andaman & Nicobar IslandsNicobar,Andaman & Nicobar,2
2,Andhra Pradesh,Adilabad,1901.0,6.725,10.488,23.288,35.560,23.119,115.546,294.119,276.865,181.615,47.310,1.339,0.000,Andhra PradeshAdilabad,Andhra Pradesh,3
3,Andhra Pradesh,Adilabad,1902.0,0.420,0.000,0.388,6.070,3.331,45.960,233.973,167.971,198.177,26.447,35.083,11.222,Andhra PradeshAdilabad,Andhra Pradesh,3
4,Andhra Pradesh,Adilabad,1903.0,6.643,1.956,0.173,4.551,33.348,132.078,436.611,334.544,226.037,138.818,14.095,8.823,Andhra PradeshAdilabad,Andhra Pradesh,3


In [11]:
state_id_map['Andaman and Nicobar'] = state_id_map['Andaman & Nicobar']
state_id_map['Dadra and Nagar Haveli'] = state_id_map['Dadra & Nagar Haveli']
state_id_map['Daman and Diu'] = state_id_map['Daman & Diu']
state_id_map['Jammu and Kashmir'] = state_id_map['Jammu & Kashmir']

In [12]:
districts['sid'] = districts['NAME_1'].apply(lambda x: state_id_map[x])

In [13]:
def state_map(month, year, state_id):
    # choose data for the year and state
    df_dynamic = df[df['Year'] == year]
    df_dynamic = df_dynamic[df_dynamic['id'] == state_id]
    # choose the map for the state
    districts_dynamic = districts[districts['sid'] == state_id]
    # name  the ID_2 column as id
    districts_dynamic = districts_dynamic.rename(columns={'ID_2': 'id'})
    # specify this id as featureid key
    districts_dynamic['id'] = districts_dynamic['id'].astype(str)
    districts_dynamic = districts_dynamic.set_index('id')
    districts_dynamic = districts_dynamic.to_json()
    districts_dynamic = json.loads(districts_dynamic)
    district_fig = px.choropleth(df_dynamic, geojson=districts_dynamic, locations='ID_2', color=month, hover_name='District')
    district_fig.update_geos(fitbounds='locations', visible=False)
    # make background transparent
    district_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    # make text white
    district_fig.update_layout(font_color='white')
    return district_fig

In [15]:
# Create the app
app = dash.Dash(__name__)

In [16]:
# Create the layout
app.layout = html.Div([
    html.H1('India District Level Data'),
    html.H2('Hover over the map to see the rainfall'),
    html.H3('Choose year'),
    dcc.Slider(
        id='year',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        # make this slider smaoother and place it in the center
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None, included=False, updatemode='drag',
        tooltip={'always_visible': True, 'placement': 'bottom'},
    ),
    html.H3('Choose month'),
    dcc.Dropdown(
        id='month',
        options=[ {'label': i, 'value': i} for i in df.columns[3:15] ],
        value='Jan'
        # make this dropdown smaller and place it in the center
        , style={'width': '40%', 'margin': 'auto'}
    ),
    html.Div([
    html.H3('India Map'),
    html.H4('Click on the state to see the district map'),
    dcc.Graph(id='india_map', figure=plot_rainfall('Jan', 2002)),
    html.H3('State Map'),
    dcc.Graph(id='district_map', figure=state_map('Jan', 2002, 'Bihar'))
    ], 
    # display the india map and state map side by side
    style={'columnCount': 2})
], style={'textAlign': 'center', 'backgroundColor':'#000000', 'color':'#ffffff'})

# Create the callback
@app.callback(
    Output('district_map', 'figure'),
    [Input('month', 'value'), Input('year', 'value'), Input('india_map', 'clickData')]
)
def update_map(month, year, state_id):
    if state_id is None:
        state_id = 10
    else:
        state_id = state_id['points'][0]['location']
        print(month, year, state_id)
    return state_map(month, year, state_id)

# callback for the india map
@app.callback(
    Output('india_map', 'figure'),
    [Input('month', 'value'), Input('year', 'value')]
)
def update_map(month, year):
    return plot_rainfall(month, year)

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2023 02:44:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 02:44:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 02:44:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 02:44:31] "POST /_dash-update-component HTTP/1.1" 200 -


Jan 2002 Uttar Pradesh


127.0.0.1 - - [20/Apr/2023 02:44:39] "POST /_dash-update-component HTTP/1.1" 200 -


Jan 2002 Bihar


...

127.0.0.1 - - [20/Apr/2023 02:45:10] "POST /_dash-update-component HTTP/1.1" 200 -


Jan 2002 Rajasthan


127.0.0.1 - - [20/Apr/2023 02:45:12] "POST /_dash-update-component HTTP/1.1" 200 -


Jan 2002 Uttarakhand


127.0.0.1 - - [20/Apr/2023 02:45:13] "POST /_dash-update-component HTTP/1.1" 200 -
